In [2]:
import pandas as pd
import os
import io
import numpy as np
import re
import time
import string
from collections import Counter, defaultdict
from sklearn import metrics
import xml.etree.ElementTree as ET
from sklearn.svm import LinearSVC
import nltk
from nltk.corpus import stopwords
import nltk
from nltk.stem.snowball import FrenchStemmer 
from nltk.corpus import stopwords
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [11]:
# Path to English Training & test data
x_path = 'cls-acl10-unprocessed/en/books/unlabeled.review'
parsed_xml = ET.parse(x_path)
x_path_test = 'cls-acl10-unprocessed/en/books/test.review'
parsed_xml_test = ET.parse(x_path_test)
# Path to french test data
x_path_test_fr = 'cls-acl10-unprocessed/fr/books/test.review'
parsed_xml_test_fr = ET.parse(x_path_test_fr)
# Path to german test data
x_path_test_de = 'cls-acl10-unprocessed/de/books/test.review'
parsed_xml_test_de = ET.parse(x_path_test_de)

def getvalueofnode(node):
    """ return node text or None """
    return node.text if node is not None else None

In [12]:
# Read english and french word2vec
def read_w2v(language='english'):
    if(language == 'french'):
        path = 'wiki.multi.fr.vec'
    elif(language == 'english'):
        path = 'wiki.multi.en.vec'
    elif(language == 'german'):
        path = 'wiki.multi.de.vec'
    t0 = time.time()
    w2v = {}
    count = 0
    with open(path, "r", encoding="utf8") as lines:
        for line in lines:
            lineArr = line.split()
            if(count!=0):
                x = []
                for value in lineArr[len(lineArr)-300:]:
                        x.append(float(value))
                w2v[' '.join(lineArr[0:len(lineArr)-300])]=  np.array(x)
            count+=1
    print(count)
    print(time.time()-t0)
    return w2v

In [13]:
def load_vec(emb_path, nmax=50000):
    vectors = []
    word2id = {}
    with io.open(emb_path, 'r', encoding='utf-8', newline='\n', errors='ignore') as f:
        next(f)
        for i, line in enumerate(f):
            word, vect = line.rstrip().split(' ', 1)
            print (word)
            vect = np.fromstring(vect, sep=' ')
            assert word not in word2id, 'word found twice'
            vectors.append(vect)
            word2id[word] = len(word2id)
            if len(word2id) == nmax:
                break
    id2word = {v: k for k, v in word2id.items()}
    embeddings = np.vstack(vectors)
    return embeddings, id2word, word2id

In [14]:
nmax = 2000000
src_path = "wiki.multi.en.vec"
embeddings, id2word, word2id = load_vec(src_path, nmax)

,
.
the
</s>
of
-
in
and
'
)
(
to
a
is
was
on
s
for
as
by
that
it
with
from
at
he
this
be
i
an
utc
his
not
–
are
or
talk
which
also
has
were
but
have
#
one
rd
new
first
page
no
you
they
had
article
t
who
?
all
their
there
been
made
its
people
may
after
%
other
should
two
score
her
can
would
more
if
she
about
when
time
team
american
such
th
do
discussion
links
only
some
up
see
united
years
into
/
school
so
world
university
during
out
state
states
national
wikipedia
year
most
city
over
used
then
d
than
county
external
m
where
will
de
what
delete
any
these
january
march
august
july
being
film
him
many
south
september
like
between
october
three
june
well
use
war
under
them
april
we
born
december
link
while
c
later
part
november
further
players
list
please
following
my
february
known
second
u
name
group
history
series
just
e
north
work
before
since
season
both
high
st
through
district
now
!
comments
because
football
music
however
diff
century
league
edits
debate
title
articles
john
same
inc

fellow
duke
tell
gallery
follow
windows
tree
hits
jazz
protection
relevant
count
situation
reviews
containing
classical
offered
com/
lady
netherlands
reports
influence
address
linear
consider
machine
domain
elements
minnesota
types
nov
serve
sydney
ministry
blood
distance
bottom
giving
boys
potential
toronto
edited
infantry
jun
formerly
oct
conflict
workers
steve
philadelphia
helped
der
nationality
dispute
scene
method
titles
berlin
conditions
arms
races
maybe
discovered
iron
extended
churches
otherwise
positive
santa
nom
longew
imperial
composed
latns
ball
width
quickly
correct
responsible
possibly
indiana
soldiers
examples
korean
timezone
genre
fish
senate
effects
gun
check
appearances
plans
renamed
sign
reporting
sweden
consists
heritage
tag
primarily
doctor
leaders
lies
inc
rivers
du
crime
liberal
stand
bob
existing
publishing
industrial
answer
split
apr
sex
mixed
acting
personnel
rail
die
premier
approach
wisconsin
sentence
root
ago
standards
comics
earned
miss
specifically
horse


discovery
explained
expedition
citation
assist
und
meanwhile
hampshire
creative
maintain
pierre
detailed
facts
frame
finance
socialist
script
camera
returns
engaged
assistance
experienced
underground
sale
beautiful
jane
abc
supposed
successor
classification
tool
mining
producing
cabinet
fr
bytes
ross
russell
citations
maintained
evening
singing
fifa
gender
venues
lakes
mail
jeff
electoral
emergency
mode
christopher
heads
proved
priest
funds
investment
romanian
session
capture
aspects
reduce
trophy
abuse
prefecture
walk
faced
normally
regarded
snow
shop
dakota
bush
coal
inhabitants
headed
gary
employees
error
invited
cable
protein
accident
decade
measure
watched
patients
downtown
animated
satellite
johnny
combination
courts
sequence
hook
clean
wed
owners
twin
distributed
describe
~
defensive
islam
photos
ottoman
trained
affected
routes
ministers
wine
elsewhere
biggest
li
lanka
carlos
landing
collected
revival
rio
communes
saturday
mps
guess
drop
sarah
laid
swimming
membership
edinburgh


ninth
arrest
choir
trials
mines
injuries
rapidly
rounds
competitive
opportunities
meetings
commented
wang
woods
exercise
jacques
objective
demolished
preferred
resort
pedro
robot
venezuela
segment
studying
edwards
aim
dancing
eagles
demonstrated
tribute
continuous
encourage
spider
acted
convinced
heroes
describing
rocks
bed
gap
reflect
mars
participating
cooperation
obtain
gothic
protest
hunting
rfa
frequent
conversion
stress
manufacturers
checkuser
voiced
innings
traditionally
jose
adventures
tiger
totally
voyage
concentration
sing
rocket
electricity
shadow
boxing
senators
doc
stanford
machines
vegas
clearer
saved
jury
calendar
noble
tommy
guilty
leo
affair
handle
extinct
responded
shares
scotia
manufacturer
tales
implementation
truck
spelling
item
load
customers
adds
spaces
cap
orphaned
ferry
prefer
push
lie
berkeley
/km²
lebanon
madison
throne
attracted
ie
lion
retrieved
manor
isip
promoting
saudi
serial
abroad
rogers
lights
grandfather
gauge
concerts
elder
reads
renaissance
uniform

temporarily
hotels
edmonton
developments
literacy
currency
missionary
arrives
hammer
dollar
ambassadors
reverts
twitter
centres
solomon
recommend
descendants
ruth
handling
customs
collect
grid
secured
certificate
destination
albania
indies
euro
consumption
feat
pushing
constantly
survivors
mansion
cardiff
temples
_name
blake
sheet
lift
confidence
cuisine
frankfurt
galaxy
ecuador
breeding
outbreak
legendary
handball
georgian
copenhagen
trek
ignored
arch
keys
proceedings
enjoy
quartet
military_history_by_country
aims
propaganda
wu
disk
realized
ne
neat
funny
punishment
accuracy
businesspeople
meter
theoretical
suspension
graduation
flew
nationalcaps
seeds
lighting
jennifer
smooth
ah
customer
armstrong
southwestern
involve
philosophical
escaped
of%
powell
kills
taste
allmusic
requiring
bros
assertion
boulevard
northeastern
brooks
sending
atomic
antarctica
strikes
reconstruction
chronicle
traveling
leslie
ellis
devon
ghana
gen
rebel
duncan
pianist
canon
nc
reformed
pack
iceland
solve
cycli

...

shadian
schurr
rikako
wearne
iyaz
murder%
speelman
/lox
applesauce
peitz
teepees
hosack
cumings
zaido
pentyl
engelke
\bobs\
bbinfrastructureinc
phantasie
caerlaverock
newtongrange
youlou
life,
viagens
hooverphonic
wode
downloaders
bullingdon
rikimaru
wentzville
kaleo
m&s
alderwood
prevc
benzion
johnsville
jaffery
tetons
voegele
fauzia
colligan
ccpa
garanti
janab
dumdum
las%
stelfox
votoms
fábián
vaisseaux
chukyo
tetrarchs
nops
redband
overmatched
ccitt
lecale
lankesh
wissmann
xaml
ediz
ratdog
pocantico
knauss
es/pub/imagenes/imagenes_
⢱alt
shameem
mandalorian
aii
grabowsky
cunedda
harsanyi
kaulitz
hochtief
emini
talita
dietician
rún
botanicals
católicos
yanick
gotvand
heathkit
semenya
yatim
jeevana
warmwater
physalus
roods
yelle
aéroports
dorabella
protium
offiah
живопись
genesys
footages
ncss
lyubimets
mcclurkin
sedol
bosso
a/h
fould
tiryns
ilgauskas
pangani
hlohovec
avola
chembl
araiza
transracial
strutters
deathblow
leering
groovies
abeysekera
hik
palomeque
rheinberg
fruiteater
heml

quelli
dombasle
ultraconservative
ltgen
mosheim
kryuchkov
endianness
definitives
paranaguá
flanner
axt
murud
mormont
gosnells
fecund
sawston
merlene
documentable
loquitur
caliendo
cppa
ashtown
fewell
uralla
nahdlatul
people_of_vietnamese_descent
murder–suicides
muskens
iltis
letterboxed
kinswoman
chaoimh
tylo
recchia
vergence
péronnas
capense
arocha
remotegoat
vågsbygd
patreon
nonwoven
cybulski
steinau
ghmyrtle
ballykelly
carotenuto
klarion
klossner
erler
poblete
harnad
crenelated
pultenaea
thesauri
foresman
ujan
vilvens
interpro
ruether
transfermarkt
azb
unloads
majoor
dishonoured
gepard
louganis
printezis
sclerite
spectrally
jenga
macneal
lasco
wattisham
yūto
tränen
halebidu
lắk
philthy
katakura
perfringens
earlville
sicurezza
wilków
pulwama
parto
chenal
jellico
ukrainy
chinchillas
chvojka
garik
episcoporum
marenghi
sphenomorphus
monomachos
dethick
juticalpa
velleius
ukmix
kawazu
\eta^
tlx
feeler
rads
humla
pungency
peces
permic
tirreno
soběslav
carcere
skk
petrojet
otira
habibur
rep

tishri
willeford
guozhong
underglaze
conduite
kvass
brulle
sfan
žukauskas
engineman
censei
politcal
content_id
lexicology
cornershop
latinist
rockfalls
weisner
vsg
yukichi
wassenaer
ladouceur
abdisho
carriles
licavoli
jordanita
hmhs
mohrenschildt
paheli
com/review
#superior
bhagavatar
radič
alzate
cavally
wildlings
siop
pirc
anitta
tuul
persuasiveness
randee
gpio
bergues
matcha
demagoguery
holts
flapped
kemmel
moortown
spendlove
mclanahan
cybersex
extensors
vuoi
postganglionic
eléctrico
tryed
hedemora
pumuckl
gravettian
pinkertons
reprintings
ossuaries
weinmann
carls
censorinus
nazrin
frisson
khateeb
narodnik
adelines
royen
cmaa
sulton
pygargus
orcid
vibhishana
%y
sconces
ukmergė
vandalous
messedrocker
irthlingborough
carterville
imperially
asgari
flatters
callcott
wetteren
brantôme
assata
etain
pecoraro
escritos
revúca
blindfolds
polsbroek
aapa
ennead
broeder
samovar
prunty
coachteams
guimaraes
éirinn
idylle
mürwik
collaboratory
stipendium
nazko
jehoram
sapone
yerger
nanostructure
ute

prejudicing
worldvision
kanbawza
zsolnay
futch
rottmann
folmar
richardi
verkehrsgesellschaft
mosser
harahan
bardonecchia
tutul
heechul
chrysanthos
fumiya
susz
giganotosaurus
ugetsu
calcineurin
rederecord
vulci
woundwort
hylobates
factsontheground
casseroles
khasso
catchiness
selick
boniek
asilomar
pothyne
august–november
sumu
∃
gryner
eynsford
lho
süleymaniye
paithan
#andante
neuinfo
tenka
vulpina
eremaea
varlam
myrtleford
zeisler
wolfswinkel
karpal
lauderhill
gekkonids
nepad
gauger
shailaja
garnham
argyriou
gotenhafen
veel
blpcrime
chandipur
sighthound
harborside
englebert
wojtowicz
cuney
bracteoles
triops
nojpetén
lanson
\bthegauntlet\
raylene
schistidium
mačvan
denio
vldl
irrealis
#wang
sonique
fb_team_templates
metaphysik
stellarkid
keko
nichita
muestra
wmar
cœurs
thaicom
busi
atikokan
kovar
но
metacognitive
gorazd
teeters
goldreich
rinella
crosslines
always_display
wahed
sensorial
fankhauser
sheens
hernanes
lochnagar
oculist
inokashira
awpbannermeta%
turnblad
oppositifolia
shutter

claudii
colab
okenia
truculent
keyloggers
titulary
forces%
bingyu
dobrescu
rubinek
agonoxenidae
wikki
keizō
wieringen
księga
playfirst
silman
goudhurst
kurtwood
london%
midir
galagos
bilaterians
statix
estola
pryer
jifu
mousseau
ialá
lời
klíma
dębnica
tennishofid
fabulae
meenu
knodel
tbmm
noeticatea
tiede
pillnitz
arhiva
grzimek
degrazia
höfner
bulganin
crummey
socioeconomically
redwine
inrockuptibles
stammheim
belene
jetstorm
pbj
schmo
dafonsoro
rampurhat
/qid
tornquist
czechoslovaks
ghirardelli
raël
spondon
riegels
codepoints
monacelli
abovian
görz
eiichiro
numantia
bruhns
τους
kainan
pattabhi
hakufu
haycraft
aleatoric
spangles
balacra
smarting
cozio
gommecourt
zhelev
fuzion
muraíle
tāzehābād
labis
weiyue
akosombo
draheim
webforum
spielvogel
beachley
com/interview
chillon
castricum
urbie
человек
naafs
sozialdemokratische
kokutai
taishanese
srei
rulesets
lemkos
christ%
lovelier
peerzada
hanshi
fideles
manyika
middlewood
encroaches
schauinsland
command/service
petrochina
miyatake
bradl

In [15]:
stopWords = set(stopwords.words('french'))
print (stopWords)

{'étais', 'étées', 'qu', 'suis', 'seraient', 'ou', 'était', 'soient', 'j', 'fussions', 'fusses', 'y', 'du', 'eût', 'ont', 'eussent', 'aura', 'serai', 'seras', 'je', 'se', 'étiez', 'pour', 'leur', 'étée', 'mon', 'avait', 'seront', 'eussiez', 'aurez', 'eûmes', 'ai', 'eûtes', 'en', 'des', 'eusse', 'ses', 'dans', 'eussions', 'ayante', 'étés', 'avez', 'as', 'c', 'ma', 'sois', 'aient', 'ayants', 'mais', 'avaient', 'aies', 'soit', 'toi', 'es', 'aurais', 'aurons', 'serez', 'son', 'eurent', 'aie', 'serait', 'eu', 'étions', 'moi', 'soyons', 'étantes', 'aurions', 'fut', 'le', 'n', 'furent', 'avions', 'étaient', 'ayez', 'lui', 'vos', 'fusse', 'eusses', 'on', 'eut', 'avec', 'nous', 'sa', 'et', 'de', 'à', 'serons', 'ayant', 'ait', 'mes', 'fûmes', 'sont', 'il', 'auraient', 'la', 'eus', 'eue', 'ton', 'aux', 'qui', 'au', 'par', 'pas', 'votre', 'ayantes', 'ces', 'été', 'étante', 'tu', 'eux', 'nos', 'est', 'sera', 'fussiez', 'd', 'notre', 'aviez', 'auront', 'même', 's', 'avais', 'auras', 'aurai', 'serion

In [16]:
stopWords = set(stopwords.words('german'))
print (stopWords)

{'über', 'dem', 'eurem', 'welche', 'keines', 'manchem', 'bei', 'anderes', 'dieselben', 'doch', 'man', 'vom', 'ist', 'jeder', 'meinem', 'und', 'selbst', 'als', 'die', 'eure', 'noch', 'vor', 'diesem', 'mein', 'einer', 'desselben', 'aber', 'wollen', 'du', 'ihres', 'kann', 'demselben', 'zur', 'deines', 'jeden', 'seines', 'welchen', 'uns', 'euer', 'während', 'seine', 'allem', 'bis', 'denn', 'keinem', 'anderen', 'nicht', 'können', 'zum', 'mir', 'meiner', 'denselben', 'war', 'gegen', 'dieses', 'hatten', 'des', 'also', 'allen', 'diese', 'einiges', 'machen', 'manchen', 'musste', 'solchen', 'deiner', 'meine', 'unseren', 'unseres', 'dieselbe', 'einen', 'diesen', 'hin', 'unserem', 'weg', 'welches', 'werden', 'von', 'wie', 'sind', 'es', 'dich', 'sollte', 'dann', 'seiner', 'jede', 'wieder', 'andern', 'was', 'dein', 'nur', 'oder', 'sein', 'dieser', 'ein', 'eine', 'euch', 'ob', 'unsere', 'nichts', 'mit', 'bin', 'einmal', 'dessen', 'ins', 'ihrer', 'seinem', 'solcher', 'derselben', 'würden', 'einig', 'z

In [17]:
keys = []
for key in word2id:
    keys.append(key)
keys

[',',
 '.',
 'the',
 '</s>',
 'of',
 '-',
 'in',
 'and',
 "'",
 ')',
 '(',
 'to',
 'a',
 'is',
 'was',
 'on',
 's',
 'for',
 'as',
 'by',
 'that',
 'it',
 'with',
 'from',
 'at',
 'he',
 'this',
 'be',
 'i',
 'an',
 'utc',
 'his',
 'not',
 '–',
 'are',
 'or',
 'talk',
 'which',
 'also',
 'has',
 'were',
 'but',
 'have',
 '#',
 'one',
 'rd',
 'new',
 'first',
 'page',
 'no',
 'you',
 'they',
 'had',
 'article',
 't',
 'who',
 '?',
 'all',
 'their',
 'there',
 'been',
 'made',
 'its',
 'people',
 'may',
 'after',
 '%',
 'other',
 'should',
 'two',
 'score',
 'her',
 'can',
 'would',
 'more',
 'if',
 'she',
 'about',
 'when',
 'time',
 'team',
 'american',
 'such',
 'th',
 'do',
 'discussion',
 'links',
 'only',
 'some',
 'up',
 'see',
 'united',
 'years',
 'into',
 '/',
 'school',
 'so',
 'world',
 'university',
 'during',
 'out',
 'state',
 'states',
 'national',
 'wikipedia',
 'year',
 'most',
 'city',
 'over',
 'used',
 'then',
 'd',
 'than',
 'county',
 'external',
 'm',
 'where',
 '

In [18]:
# tokenization of input text
def preprocess_tokenize(doc, keep_internal_punct=True):
    doc = doc.lower()
    if(keep_internal_punct==False):
        return np.array(re.findall("[\w]+", doc))
    else:
        token_list = doc.split()
        tokenized_list=[]
        for token in token_list:
            new_token=''
            for i in range(0,len(token)):
                if(token[i] not in string.punctuation):
                    new_token+=token[i]
            tokenized_list.append(new_token)
        return tokenized_list

In [19]:
# Stop word removal of tokenized input data
def stopRemoval(tokenized_input, lang):
    ## Removal of Stop words
    if lang == 'english':
        stopWords = set(stopwords.words('english'))
    if lang == 'french':
        stopWords = set(stopwords.words('french'))
    if lang == 'german':
        stopWords = set(stopwords.words('german'))
    
    wordsFiltered = []
    for w in tokenized_input:
        if w not in stopWords:
            wordsFiltered.append(w)
    return wordsFiltered

In [20]:
# Stop word removal of tokenized input data
from nltk.stem import WordNetLemmatizer

def get_lemmatized_text(tokenized_review):
    lemmatizer = WordNetLemmatizer()
    lemmatizedStr = []
    for word in tokenized_review:
        lemmatizedStr.append(lemmatizer.lemmatize(word))
    return lemmatizedStr

In [21]:
import math
def cosine_similarity(v1,v2):
    "compute cosine similarity of v1 to v2: (v1 dot v2)/{||v1||*||v2||)"
    sumxx, sumxy, sumyy = 0, 0, 0
    for i in range(len(v1)):
        x = v1[i]; y = v2[i]
        sumxx += x*x
        sumyy += y*y
        sumxy += x*y
    return sumxy/math.sqrt(sumxx*sumyy)

v1,v2 = embeddings[12311],embeddings[20334] 
print(cosine_similarity(v1,v2))

0.7850404957232481


In [22]:
word2id['kidnap']

20334

In [23]:
w2v_fr = read_w2v('french')

200001
28.660369157791138


In [24]:
w2v = read_w2v('english')

200001
28.330248832702637


In [25]:
w2v_de = read_w2v('german')

200001
30.164345741271973


In [26]:
# Parsing the traing data from xml and pre-processing it 
X = []
y = []
X_refer = []
for node in parsed_xml.getroot():
    summary = node.find('summary')
    rating = node.find('rating')
    text = node.find('text')
    tokens_summary = getvalueofnode(summary)
    tokens_text = getvalueofnode(text)

#    sr = stopRemoval([*preprocess_tokenize(tokens_summary), *preprocess_tokenize(tokens_text)], 'english')
#    lemmaStr = get_lemmatized_text(sr)
    lemmaStr = get_lemmatized_text([*preprocess_tokenize(tokens_summary), *preprocess_tokenize(tokens_text)])
    X.append(lemmaStr)

#    X.append([*preprocess_tokenize(tokens_summary), *preprocess_tokenize(tokens_text)])
    X_refer.append(tokens_summary + " " + tokens_text)
    if(float(getvalueofnode(rating))>3):
        y.append('positive')
    else:
        y.append('negative')

In [27]:
# Parsing the test data from xml and pre-processing it 
X_test = []
y_test = []
X_test_refer = []
for node in parsed_xml_test.getroot():
    summary = node.find('summary')
    rating = node.find('rating')
    text = node.find('text')
    tokens_summary = getvalueofnode(summary)
    tokens_text = getvalueofnode(text)
    
#     sr = stopRemoval([*preprocess_tokenize(tokens_summary), *preprocess_tokenize(tokens_text)], 'english')
    lemmaStr = get_lemmatized_text([*preprocess_tokenize(tokens_summary), *preprocess_tokenize(tokens_text)])
    X_test.append(lemmaStr)

#    X_test.append([*tokenize(tokens_summary), *tokenize(tokens_text)])
    X_test_refer.append(tokens_summary + " " + tokens_text)
    if(float(getvalueofnode(rating))>3):
        y_test.append('positive')
    else:
        y_test.append('negative')

In [28]:
# Using TFIDF vectorizer matrix and word2vec vector, we create TfidfEmbeddingVector matrix 

from sklearn.feature_extraction.text import TfidfVectorizer

class TfidfEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.word2weight = None
        self.dim = len(next(iter(word2vec.values())))

    def fit(self, X):
        tfidf = TfidfVectorizer(analyzer=lambda x: x)
        tfidf.fit(X)
        max_idf = max(tfidf.idf_)
        self.word2weight = defaultdict(
            lambda: max_idf,
            [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()])

        return self

    def transform(self, X):
        np_ar = np.array([
                np.mean([self.word2vec[w] * self.word2weight[w]
                         for w in words if w in self.word2vec] or
                        [np.zeros(self.dim)], axis=0)
                for words in X
            ])
        return np_ar

In [29]:
#Creating TfidfEmbeddingVector of English train data and Fitting the data into SVM Classification model 

vectorizer = TfidfEmbeddingVectorizer(w2v)
vectorizer.fit(X)
X_t = vectorizer.transform(X)
clfLSVC = LinearSVC()
LSVC = CalibratedClassifierCV(clfLSVC)
LSVC.fit(X_t, y)

C:\Users\Saurabh\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


CalibratedClassifierCV(base_estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
            cv='warn', method='sigmoid')

In [30]:
X_t

array([[-0.05391549, -0.0181576 , -0.14784638, ...,  0.1448012 ,
         0.03826498,  0.01422183],
       [-0.05279255, -0.02801592, -0.1094643 , ...,  0.15838439,
         0.10243242,  0.02701922],
       [-0.08818437, -0.02670984, -0.11644781, ...,  0.11835812,
         0.10991745,  0.04395713],
       ...,
       [-0.02400398, -0.00763684, -0.14029693, ...,  0.13018747,
         0.11367567,  0.02697002],
       [-0.04355034,  0.04039371, -0.12856679, ...,  0.15803848,
         0.13744645, -0.02189843],
       [-0.08281479,  0.00197197, -0.09820781, ...,  0.09932802,
         0.10626917,  0.00976946]])

In [31]:
# Creating TfidfEmbeddingVector of English testing data and predict on SVM classifier 

vectorizer_test = TfidfEmbeddingVectorizer(w2v)
vectorizer_test.fit(X_test)
X_t_test = vectorizer_test.transform(X_test)
result = LSVC.predict(X_t_test)
pred_prob = LSVC.predict_proba(X_t_test)
metrics.accuracy_score(y_test, result)

0.843

In [32]:
y_test = np.asarray(y_test)
misclassified = np.where(y_test != result)
misclassified
len(misclassified[0])

314

In [33]:
# confusion_matrix(y_test, result)
print(classification_report(y_test, result))

              precision    recall  f1-score   support

    negative       0.83      0.86      0.85      1000
    positive       0.85      0.83      0.84      1000

   micro avg       0.84      0.84      0.84      2000
   macro avg       0.84      0.84      0.84      2000
weighted avg       0.84      0.84      0.84      2000



In [34]:
print ('Accuracy Score ', metrics.accuracy_score(y_test, result))
print ('F1 Score ',metrics.f1_score(y_test, result,average="binary", pos_label="negative"))
print ('Precision Score ',metrics.precision_score(y_test, result,average="binary", pos_label="negative"))
print ('Recall Score ',metrics.recall_score(y_test, result,average="binary", pos_label="negative"))

Accuracy Score  0.843
F1 Score  0.8451676528599606
Precision Score  0.8336575875486382
Recall Score  0.857


In [98]:
# Error analysis by examining misclassified review
# Get Misclassified review

misclass_index = []
max_pred_prob = {}
for i in range(0,len(X_test)):
    if y_test[i] != result[i]:
        misclass_index.append(i)
        max_pred_prob[i] = max(pred_prob[i])

max_pred_prob = sorted(max_pred_prob.items(), key=lambda x: -x[1])
#print (max_pred_prob)
print (len(max_pred_prob))

count = 0
for (k,v) in max_pred_prob[:]:
    if v < 0.6:
        count = count + 1
        print (X_test_refer[k], y_test[k], result[k], v)
        #print (y_test[k], result[k], v)
        print ('\n')
        words = re.sub("[^\w]", " ",  X_test_refer[k]).split()
        word_count = 0
        for i in range(len(words)):
            if words[i] in keys:
                word_count = word_count + 1
        print(word_count)
        print ('\n')
print (count)

339
A psycological mishap
 Though this novel is supposed as a psycological materpiece I have found it to be sordid and incongruent. Stendhal should have spent less time on his discriptions of Julien Sorel's superficial love endeavors and  &quot;rendez-vous&quot; and more time on his character's true personality.  After reading 508 pages I found the main character to be a complete  stranger. All that was revealed about Julien was that he stuck to his  morals all the way to the end. Stendhal should have learned from his  creation's ideals and stuck to the basics
 negative positive 0.5995180950510522


82


Ten weird, surprising, tense, comical, and often unforgettable stories
 A family on vacation encounters a cold-blooded gang, a gullible and naive housewife is struck by a mysterious (but hilariously common) "illness," a 104-year-old Civil War veteran is a featured guest at his 62-year-old daughter's high school graduation--each of O'Connor's stories portray characters in improbable, bi

54


SOME THINGS TASTE GOOD BUT BE WARNED!
 Well I bought this book hoping that I could have more breakfast ideas especially now that my children are going to school full time and I dislike processed cereals. I am a domestic engineer who has studied culinary arts specializing in pasteries. All I have to say is that the recipes that I have tried taste alright but the timing is off. Some pancake recipes call for a little much baking powder that gives the hotcakes a soapy flavour
 negative positive 0.5902767840302955


68


The best inn ever
 This book is so funny because of the character's attitude. It even has strong words, like "makintosh".But I like it best because it has poems that rhyme
 positive negative 0.5899009748832272


27


This Book Is A Must Read If You Want To Deeply Understand Persuasion
 This is a book that you'll read over and over. You'll dog ear the pages and turn them yellow with highlighter. There is pure gold to be mined on every page.
 
 What makes this book so va

230


An 18th Century essential
 I agree with the reviewers who've described this as a slow read. No argument there. But it's also a very important 18th Century work and one essential to anyone interested in the literature of the period.  If you read the book with an ironical slant it's much easier to see that there is actual humor in the often improbable situations. Granted, it may not be laugh out loud humor, but it is there. The poor vicar is modeled on the biblical Job, to a very exaggerated extent.  It's so exaggerated that the modern reader will likely be rolling his eyes, I know.  If you're looking for tight plot and fast pace this won't be your book, but those reading 18th Century literature will realize the novel was much different at the beginning that it is now.  You don't read THE VICAR OF WAKEFIELD for the plot. You read it for the study.  It remains an important book
 positive negative 0.5876787605134873


144


He is so smart
 Modern artists are often attacked because on

113


Unfortunately boring
 I'm a very spiritual person and I was expecting this CD to provide daily inspiration and help me to remember the very few things that are important in life.  Unfortunately, for me at least, the CD is rather boring and not very inspirational.  I'm about 30 minutes in to it and have decided to stop listening to it.  Fortunately I have a Christian radio station that does a lot more for me.
 negative positive 0.5781352948192693


65


Amara awakenes
 The late Richard Laymon's book "To Wake the Dead" is about a mummy who comes to life to terrorize the Los Angeles community that is housing the Egyptian exibit.  Also there is a strong side story about people being kidnapped and held as sex slaves somewhere.  That is actually the creepier story.  But the mummy one is really a dandy too.  The Mummy story is not my favorite horror theme (and I did not like the Stephen Sommers/Brenden Frasier movies, either), but then it had never been presented like this.  Actually a 

54


Environmental Planning
 I am a student in environmental planning and have found this book to be extremely useful.  It address a variey of things such as watershed, storm discharge, streamflow, etc. It really gives the reader an idea of how to design with nature
 positive negative 0.5674027726832528


38


Roland Martins 101 bass catching secrets
 This is one of the best books i have ever read. The information is presented in an easy to understand formate.  It was infromative yet easy to read. I would recomend this book to anyone that would liketo learn more about bass
 positive negative 0.5670739382167996


39


STUNNED (in a good way)
 First, let me say that I am a very independent, very headstrong, liberal wife who has a higher education than her husband (masters' degree), owns a business and takes no B.S. from anyone.  When this book came out, I truly thought it was a joke.  WHO WOULD WRITE A BOOK LIKE THIS OUTSIDE OF THE 1950s?!  And WHERE IS THE "PROPER CARE... OF WIVES"?!  T

...

320


A brilliant but flawed man
 After watching the film, "Finding Neverland", I was curious to learn more about the fates of the 5 boys orphaned in the story.  Although Barrie did not gain custody of the boys, he did maintain a strong presence in their lifes. In fact, he did much more than just keep an eye on them over the years.  He showered them with gifts; traveled with them; and of course, financially helped them.  He seemed particularly obsessed with the older boy, George.  Barrie loved to photograph all of the boys and some of the photos included in this book show the children naked.  Was Barrie a pedophile of sorts?  None of the boys ever accused him of such.  And Barrie's love and concern for the boys did not end when they became men who were tall enough to tower over the diminutive playwright.  
 
 This is a comprehensive look at Barrie's life without passing any judgement.  That is left to the reader.  And after reading this book, I have to say that I'm STILL unable to figu

97


A return to roots
 As good as it was when first published...even more meaningful after 30 years. As valid now as it was then
 positive negative 0.5293694796780367


21


DETAILED, INTERESTING AND FUN HOW-TO BOOK!!
 This book is not only an interesting read, but fun as well!  The pictures detail everything from each step of refurbishing a sailboat, including what tools to use and how to use them.  I just ordered 3 more copies and can't wait to give them to my boating enthusiastic friends!  Whether your re-doing a sailboat or not, I would highly recommend this to anybody who's into boating.
 positive negative 0.5288571128756602


64


Barry outdoes his previous form
 Dave Barry writes as though he were Elmore Leonard on speed.  It is hard to say exactly where the plot loses and contact with reality and possibility, but at the end you have to sit back and laugh and blow a sigh of satisfaction and relief that such things cannot happen, such people cannot actually exist.  Quite.  Like 

159


Excellent, practical advice
 I made the mistake of buying the first edition of this book (published in 2001), but found it to be an excellent guide to the legal issues involved with public domain textual material.  I have already published one book which relied heavily on public domain materials, and am working on a second, and when I bought this book I had several specific questions that I was trying to answer.  This book answered all of them, as well as some that I hadn't thought of!  I'll probably end up buying the second edition, just to see what has changed in the last few years
 positive negative 0.5242169992413487


98


A Real Classic
 When I was growing up, my Mother told me that she had been given this book by a beau, had read it and enjoyed it. This was in the early 1900s! When I saw it in a used Book Store about a year ago, I picked it up. I am just now  reading it. I was totally surprised to find that it is beautifully written, presents a wonderful picture of a vanis

112


Poor analogies, and no real argument
 Vanishing Voices does a good job of showing how larger languages are destroying smaller ones, and the methods of language death.  This is all pretty much common knowledge.  However, the authors fail in their attempt to give a reason as to WHY language death is something with which we should be concerned.  The only argument they put forth is in a ecological/enviromental analogy, which says that biological diversity is good and stable, therefore, linguistic diversity must.  However, they only go part way in their analogy and reject natural selection for languages.  They also show that linguistic diversity corresponds to environmental diversity, but state this has nothing to do with the inexcessiblity of the areas.  The violin-playing and loaded words are hard to stomach if you are looking for good social science.  I would not suggest the book unless you are an ecological activist wanting to try to link your cause with "saving cultures" or with 

574


If you are interested in Early American Furniture, read this
 Having been sent this book as a present by a friend in the US, I felt I must tell others just what a treasure trove it is. It has been written by  someone who not only obviously loves his subject, but who has written a  clear, easy-to-read amd well-illustrated guide for those of us who have  developed an interest in early American furniture but who lacked expertise.  It also tells you how to care for furniture. I can't recommend it highly  enough
 positive negative 0.5010938241446601


84


My Last Reading of a JAK
 Why do I feel like the serpent in the Garden of Eden?... Sorry, I really liked "Deep Waters", but I probably won't buy another JAK unless I am stuck at an airport and I have read every other available Romance, Mystery or Thriller on the newsstand.  I think I have made myself clear.  It is time for me to find some new talented authors.  Although I know I have some previously read titles of hers to review.  H

In [99]:
# Parsing french test data from xml and pre-processing it 

X_test_fr = []
y_test_fr = []
X_test_refer_fr = []
for node in parsed_xml_test_fr.getroot():
    summary = node.find('summary')
    rating = node.find('rating')
    text = node.find('text')
    tokens_summary = getvalueofnode(summary)
    tokens_text = getvalueofnode(text)
    
#     sr = stopRemoval([*preprocess_tokenize(tokens_summary), *preprocess_tokenize(tokens_text)], 'french')

    lemmaStr = get_lemmatized_text([*preprocess_tokenize(tokens_summary), *preprocess_tokenize(tokens_text)])
    X_test_fr.append(lemmaStr)
    
#    X_test_fr.append([*preprocess_tokenize(tokens_summary), *preprocess_tokenize(tokens_text)])
    X_test_refer_fr.append(tokens_summary + " " + tokens_text)
    if(float(getvalueofnode(rating))>3):
        y_test_fr.append('positive')
    else:
        y_test_fr.append('negative')

In [100]:
# Creating TfidfEmbeddingVector of french testing data and predict on SVM classifier 

vectorizer_test_fr = TfidfEmbeddingVectorizer(w2v_fr)
vectorizer_test_fr.fit(X_test_fr)
X_t_test_fr = vectorizer_test_fr.transform(X_test_fr)
result = LSVC.predict(X_t_test_fr)
metrics.accuracy_score(y_test_fr, result)

0.758

In [101]:
y_test_fr = np.asarray(y_test_fr)
y_test_fr
misclassified_fr = np.where(y_test_fr != result)
# misclassified
len(misclassified_fr[0])

484

In [102]:
print(classification_report(y_test_fr, result))

              precision    recall  f1-score   support

    negative       0.77      0.73      0.75      1000
    positive       0.75      0.78      0.76      1000

   micro avg       0.76      0.76      0.76      2000
   macro avg       0.76      0.76      0.76      2000
weighted avg       0.76      0.76      0.76      2000



In [103]:
print ('Accuracy Score ', metrics.accuracy_score(y_test_fr, result))
print ('F1 Score ',metrics.f1_score(y_test_fr, result,average="binary", pos_label="negative"))
print ('Precision Score ',metrics.precision_score(y_test_fr, result,average="binary", pos_label="negative"))
print ('Recall Score ',metrics.recall_score(y_test_fr, result,average="binary", pos_label="negative"))

Accuracy Score  0.758
F1 Score  0.7515400410677618
Precision Score  0.7721518987341772
Recall Score  0.732


In [104]:
# Error analysis by examining misclassified review
# Get Misclassified review

misclass_index = []
max_pred_prob = {}
for i in range(0,len(X_test_fr)):
    if y_test_fr[i] != result[i]:
        misclass_index.append(i)
        max_pred_prob[i] = max(pred_prob[i])

max_pred_prob = sorted(max_pred_prob.items(), key=lambda x: -x[1])
#print (max_pred_prob)
print (len(max_pred_prob))

count = 0
for (k,v) in max_pred_prob[:]:
    if v < 0.6:
        count = count + 1
        print (X_test_refer_fr[k], y_test_fr[k], result[k], v)
        #print (y_test[k], result[k], v)
        print ('\n')
        words = re.sub("[^\w]", " ",  X_test_refer_fr[k]).split()
        word_count = 0
        for i in range(len(words)):
            if words[i] in keys:
                word_count = word_count + 1
        print(word_count)
        print ('\n')
print (count)

484
Coben fait du Coben Sympa mais sans plus. A force de toujours prendre à peu près les mêmes sujets, on commence à voir les grosses ficelles. Sans surprise. negative positive 0.5966665662946667


20


Techniques traditionnelles de palissades Livre très précis sur les différentes techniques traditionnelles de construction de palissades, bordures, tressages divers. Beaucoup de références aux techniques japonaises avec leurs noms japonais. Nombreux croquis explicatifs et photos de palissades dans des jardins japonais. positive negative 0.596521617474152


24


LE HÉRISSON PENSANT Avec une sensibilité aiguë et beaucoup d'esprit, MB nous donne une image amusante de certains milieux francais, tout au long d'un récit qui a des allures de conte de fée, centré sur un personnage original : Madame Michel, concierge d'un immeuble bourgeois parisien et passionnée de lecture. Sur cette base, j'ai adopté L'ÉLÉGANCE DU HÉRISSON dans le cercle de lecture que j'anime à Stockholm - une douzaine de Suéd

72


Surprenant ! Les lecteurs de ce livre seront d'accord avec moi pour affirmer qu'ils ne consideront plus jamais leur ordinateur comme avant ! positive negative 0.5577898124372833


16


Une traduction plus que décevante. Bien que l'oeuvre de Jordan soit toujours aussi bonne, je déconseille fortement ce tome aux fans. Si vous en avez la possibilité lisez plutôt la VO (également dispo sur Amazon), ou bien la traduction espagnole.
 Bien que la traduction des tomes précedents par Arlette Rosenblum, chez Rivages, laissait à désirer sur certains points, celle de Fleuve Noir est proprement scandaleuse. La traductrice introduit des faux sens. du type "les Aiels portent des épées courtes" (au lieu de lances), ce qui est contraire à leur culture! Les "guerres de l'eau" se transforment en "brouilles maritimes"... étrange pour un peuple qui vit dans le désert, non?  Les Amis du ténébreux deviennent "Elus du ténébreux", ce qui entraine une confusion avec les réprouvés, également appelés Elus. L

173


Manque de structure et de directions Avec plus de 2 voyages en Sardaigne (Nord-Sud-Est-Ouest) je trouve que ce livre n'est pas intéressant. Il n'y a pas de directions vers les différents lieux ou vraiment très partiel (ex. liste des plages mais pas de localisation géographique). Les descriptions y sont vraiment minimum. 
 Nous avons trouvé un ouvrage beaucoup plus intéressant sur place avec les différents lieux à visiter qui sont détaillés. negative positive 0.5338656768278369


53


Courage sans révélations Un ouvrage intéressant sur le fond bien que ne révélant pas grand chose à ceux qui ont déja cotoyé l'univers carcéral ou simplement lu sur la question.
  Sur la forme c un catalogue d'anecdotes crues mais ô combien réalistes.
  Un livre qui vaut surtt d'être lu pour honoirer le courage de son auteur.
  A conseiller. positive negative 0.5319935394187126


43


du remaché une intrigue linéaire. une pelotte de grosses ficelles bien éculées. un flot de bon sentiments sous un vern

0


désespérant La contrainte formelle (2 pages) pour ces 500 portraits ou fictions atteint la prouesse technique , l'écriture chez Jauffret est toujours parfaitement maîtrisée. Mais l'ouvrage suscite assez rapidement l'accablement et le désespoir, par l'accumulation de tant de noirceurs, de perversités, d'inhumanité. Il ne nous sera jamais offert la grâce, ou le répit, d'une mise à distance, d'un sourire, d'une émotion ; les 500 micro chapitres sont autant de fenêtres sur un monde froid, absurdement cruel, et on finit par s'interroger sur les motivations de l'auteur. Lisons de préférence les Vies minuscules de Michon, qui nous réconcilie avec une humanité simple et quotidienne. negative positive 0.5007345482929818


75


46


In [105]:
# Parsing german test data from xml and pre-processing it 

X_test_de = []
y_test_de = []
X_test_refer_de = []
for node in parsed_xml_test_de.getroot():
    summary = node.find('summary')
    rating = node.find('rating')
    text = node.find('text')
    tokens_summary = getvalueofnode(summary)
    tokens_text = getvalueofnode(text)
    
#     sr = stopRemoval([*preprocess_tokenize(tokens_summary), *preprocess_tokenize(tokens_text)], 'french')

    lemmaStr = get_lemmatized_text([*preprocess_tokenize(tokens_summary), *preprocess_tokenize(tokens_text)])
    X_test_de.append(lemmaStr)
    
#    X_test_fr.append([*preprocess_tokenize(tokens_summary), *preprocess_tokenize(tokens_text)])
    X_test_refer_de.append(tokens_summary + " " + tokens_text)
    if(float(getvalueofnode(rating))>3):
        y_test_de.append('positive')
    else:
        y_test_de.append('negative')

In [106]:
# Creating TfidfEmbeddingVector of german testing data and predict on SVM classifier 

vectorizer_test_de = TfidfEmbeddingVectorizer(w2v_de)
vectorizer_test_de.fit(X_test_de)
X_t_test_de = vectorizer_test_de.transform(X_test_de)
result = LSVC.predict(X_t_test_de)
metrics.accuracy_score(y_test_de, result)

0.734

In [107]:
y_test_de = np.asarray(y_test_de)
y_test_de
misclassified_de = np.where(y_test_de != result)
# misclassified
len(misclassified_de[0])

532

In [108]:
print(classification_report(y_test_de, result))

              precision    recall  f1-score   support

    negative       0.68      0.90      0.77      1000
    positive       0.85      0.57      0.68      1000

   micro avg       0.73      0.73      0.73      2000
   macro avg       0.76      0.73      0.73      2000
weighted avg       0.76      0.73      0.73      2000



In [109]:
print ('Accuracy Score ', metrics.accuracy_score(y_test_de, result))
print ('F1 Score ',metrics.f1_score(y_test_de, result,average="binary", pos_label="negative"))
print ('Precision Score ',metrics.precision_score(y_test_de, result,average="binary", pos_label="negative"))
print ('Recall Score ',metrics.recall_score(y_test_de, result,average="binary", pos_label="negative"))

Accuracy Score  0.734
F1 Score  0.7708871662360034
Precision Score  0.6770045385779122
Recall Score  0.895


In [110]:
# Error analysis by examining misclassified review
# Get Misclassified review

misclass_index = []
max_pred_prob = {}
for i in range(0,len(X_test_de)):
    if y_test_de[i] != result[i]:
        misclass_index.append(i)
        max_pred_prob[i] = max(pred_prob[i])

max_pred_prob = sorted(max_pred_prob.items(), key=lambda x: -x[1])
#print (max_pred_prob)
print (len(max_pred_prob))

count = 0
for (k,v) in max_pred_prob[:]:
    if v < 0.6:
        count = count + 1
        print (X_test_refer_de[k], y_test_de[k], result[k], v)
        #print (y_test[k], result[k], v)
        print ('\n')
        words = re.sub("[^\w]", " ",  X_test_refer_de[k]).split()
        word_count = 0
        for i in range(len(words)):
            if words[i] in keys:
                word_count = word_count + 1
        print(word_count)
        print ('\n')
print (count)

532
absolut nicht hilfreicht! leider gibt das buch keinerlei praktische tips, wie man selbst most herstellen kann. dieser bereich beschränkt sich auf einige wenige seiten. die information, die man dort findet, findet man innerhalb weniger minuten im internet. es sind keinerlei praktische tips oder vorschläge zu finden! negative positive 0.5986796421203182


34


Darüber sollte jeder Hundehalter Bescheid wissen In diesem Buch werden alle möglichen Notfallsituationen detailliert aber trotzdem leicht verständlich dargestellt und erklärt. Die notwendigen Handgriffe und erste Versorgungsmaßnahmen sollte jeder Hundehalter beherrschen um seinem oder einem anderen Hund im Ernstfall helfen zu können. Deshalb reicht es nicht aus das Buch nur einmal zu lesen. Der Inhalt sollte wirklich gelernt werden. positive negative 0.5953333002272893


28


Praxisgerecht für DSLR-Einsteiger In leicht nachvollziehbaren, übersichtlichen Schritten werden die Kamerafunktionen der Sony alpha-Einsteigerserie verstä

106


Sehr gut recherchiert! Und bewegend! Wenn ich mir die Kritiken so durchlese und die erteilten Sterne zähle und die abgegebenen Stimmen, so wird klar, daß fast alle Leser - und hier schließe ich mich mit ein - das Buch fantastisch finden.
 Jan van Helsing hat nicht nur in seinen verbotenen Büchern, die ja immerhin fast zweihunderttausend mal innerhalb zweier Jahre über den Ladentisch gingen und auf der "Spiegel"-Bestsellerliste Erwähnung fanden, dargelegt, daß auf unserem Planeten nicht alles mit rechten Dingen zugeht, sondern sehr mächtige Organisationen und Familien im Hintergrund der Weltgeschichte ein eigenes Programm durchziehen - die Neue Weltordnung. 
 Jan van Helsing als berühmtester Autor, und gleichzeitig auch profundester Kenner dieser Materie, hat fleißig recherchiert und solide begründet (sei erstes Buch hat dreißig Seiten Quellennachweise und weiterführende Literaturhinweise aufgeführt!!!), was heute immer offensichtlicher wird - nämlich, daß wir keinesfalls in einer

50


Witzig und spannend! Als ich mir vor zwei Jahren die Augen operieren ließ und nicht lesen durfte habe ich das Hörbuch entdeckt und bin gleich auf die genialen Werke von Walter Moers gestossen, die durch die Stimme von Dirk Bach noch gewinnen. Ensel und Krete war das erste "Werk" von Moers/Bach, das ich mir angehört habe und ich war hin- und weg. Witzig, detailreich und spannend. Obwohl ich inzwischen wieder lesen darf, habe ich mir sämtliche Moers-Hörbücher zugelegt und meine Begeisterung ist ungebrochen. positive negative 0.5723990403844045


47


Ein spitzen Buch!! Ich kann allen, die Bücher über Geheimbunde mögen, dieses Exemplar wärmstens empfehlen. Ich selbst habe dieses Buch auf 2 Tage ausgelesen u. hoffe nun auf irgendeine Art von Fortsetzung..... positive negative 0.5704365003591437


13


sehr nah am Ideal Natürlich kann man die Dummheit des Erzählers kritisieren. Oder kann einem die düstere, selbstzerstörische Grundhaltung des Romans nicht gefallen.
 Aber es gibt nur gan

15


Spannender Blick hinter die Kulissen und amüsant geschrieben Wer die Ironie zwischen den Zeilen nicht bemerkt, ist selbst Schuld! Ich bin kein Glamour-Freak, aber eben auch nicht von gestern, und was in BACKSTAGE seht, ist wirklich super interessant und als Leser kommt man voll auf seine Kosten: Beispielsweise geht es unter anderem um den immensen Druck auf Grund von Maßevorgaben, um Mädchen, die in O-Saft getränkte Watte essen, um schlank zu bleiben oder aber gar nichts essen und sich Poster von reichhaltigen Torten, etc. ins Zimmer hängen und anbeten wie einen Rockstar. Man liest ja immer wieder Schlagzeilen zum Thema, doch es von einer Insiderin zu hören, ist noch einmal etwas anderes.
 Doch es gibt auch Witziges, wie etwa die Backstage-Anekdoten bei Laufstegschauen und Shootings. Köstlich amüsiert habe ich mich auch bei den Anekdoten über vermasselte Dates oder über männliche Mitbewohner mit Brustimplantaten, um sich somit das Fitness-Studio zu ersparen, oder auch bei Storys ü

266


Kurzweiliger Einblick ins Beraterleben Wer sich bereits mit dem Beratungsgeschäft befasst hat, wird sich  in vielen Punkten bestätigt finden. Wer noch nichts über den Berateralltag weiß, findet eine recht vollständige Darstellung eines möglichen Arbeitslebens in einer großen Unternehmensberatung. Nicht alle Seiten werden beleuchtet, doch wird vieles, z.T. in wirklich erfrischend amüsierender Weise erwähnt. Wünschenswert wäre so manches Mal eine ausführlichere Darstellung der Arbeitstage selbst gewesen, so überwiegen eher die Aktivitäten davor und danach. Schön sind die gelegentlichen Einschübe der Gedanken des Autors, speziell im Hinblick auf vorher, nachher Ansichten. Insgesamt ein kurzweiliges Werk, dass man aber eher lesen sollte, wenn man sich ernsthaft für diesen Beruf interessiert. positive negative 0.540200936970836


49


Lesenswert, erschütternd und wohltuend sachlich Litwinenko wachte über das, was hinter russischen Fassaden vorgeht, und er scheute keine Mühe, seine Erk

92


Spirituelle 'Suche' ? Wozu? Man ISTschon frei! Der Autor stellt einem mit diesem umwerfenden Buch eine andere Möglichkeit des Erwachens vor: Er landet in einem Gefängnis und die Umstände sind so elend, dass sein Verstand einfach aufgibt und das Ego verschwindet. Einfach so. Und genau das macht dieses Buch aus: Es zeigt einem auf, dass keine Übung oder spirituelle Praxis notwendig ist, um zu erkennen, wer oder was man wirklich ist. Mit viel Humor wird der kosmische Witz erklärt, dass man glaubt, man müsse oder könne befreit werden. Die verschiedenen Facetten der Blindheit für das eigene Sein werden erzählt und man erkennt - zumindest verstandesgemäß - sofort, dass man tatsächlich immer und JETZT schon frei ist. Es gibt also nichts zu tun, als sich zurück zu lehnen und das Leben zu genießen. Dieses Buch kann einem außerdem helfen, widrige Lebensumstände zu akzeptieren und sie als (quasi selbstgewählte) ‚göttliche Therapie' zu erkennen, die eben manchmal das Ego einfach zur Aufgabe z

163


wunderbarer 2. Teil der Sterling- Familienreihe Nachdem sie in ihrer ersten Ballsaison schon drei Verehrer um ihre Hand abweist, bekommt Lady Arabella vom ton" den nicht sehr schmeichelhaften Beinamen die Unerreichbare" verpasst. Die rothaarige, temperamentvolle Arabella, Tochter eines Vikars, ärgert sich sehr über diesen Beinamen, doch wie es der Zufall so will, bekommt sie eines Abends auf einem Ball einen weiteren Antrag, den sie wieder einmal ablehnen will. Während ihres Gesprächs, wird sie von Sir Justin Sterling, einem Herzensbrecher und Lebemann belauscht. Justin und sein Freund haben, angestachelt durch fünf weitere Gentlemens des tons" eine private Wette abgeschlossen. Wenn es Justin gelingt, Lady Arabella innerhalb eines Monats zu verführen, bekommt er von seinem Freund Gideon sechstausend Pfund in bar ausbezahlt. Obwohl sich Justin anfangs nicht wohl in seiner Haut fühlt, nimmt er die Wette jedoch an und staunt nicht schlecht, als er Arabella schließlich persönlich geg